In [1]:
import requests
from dotenv import load_dotenv
import os
from dataclasses import dataclass, field
from typing import Optional, Dict, Any, List
import json
from datetime import datetime
from pprint import pprint
from urllib import parse

os.makedirs("data", exist_ok=True)
os.makedirs("data/raw", exist_ok=True)

In [2]:
load_dotenv()

@dataclass
class BasicConfig:
    api_key: str = field(default_factory=lambda: os.getenv("GOOGLE_API_KEY", "INVALID"))
    base_url: str = field(default="https://www.googleapis.com/youtube/v3")
    if api_key == "INVALID":
        raise ValueError("GOOGLE_API_KEY environment variable is not set or invalid.")


    

In [ ]:
# Exploring responses

urldefaults = BasicConfig()
def get_url(endpoint: str, basic_config: BasicConfig, params: Optional[Dict[str, str]] = None) -> str:
    if params is None:
        params = {}
    params['key'] = basic_config.api_key
    url = f"{basic_config.base_url}{endpoint}?{parse.urlencode(params)}"
    return url

temp_dir = "responses/raw/colab/"

def get_response(endpoint: str, params: Optional[Dict[str, str]] = None, dump: bool = False, max_pages: int = 1, base_file_name: Optional[str] = None, basic_config: BasicConfig = BasicConfig()) -> List[Dict[str, Any]]:
    results = []
    page_count = 0
    next_page_token = None

    if base_file_name:
        base_name_for_dump: str = temp_dir + base_file_name.replace('/', '_')
        path = os.path.dirname(base_name_for_dump)
        os.makedirs(path, exist_ok=True)
        base_name_for_dump = f"{base_name_for_dump}_response"

    else:
        base_name_for_dump: str = f"{temp_dir}{endpoint.replace('/', '_')}_response"
        os.makedirs(os.path.dirname(base_name_for_dump), exist_ok=True)


    while page_count < max_pages:
        if next_page_token:
            params = params.copy() if params else {}
            params['pageToken'] = next_page_token
        url = get_url(endpoint, basic_config, params)
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Error: {response.status_code} - {response.text}")
        data = response.json()
        results.append(data)

        if dump:
            print(f"Dumping page {page_count + 1}...")
            page_suffix: str = f"_page_{page_count+1}"
            timestamp: str = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
            current_file_name: str = f"{base_name_for_dump}{page_suffix}_{timestamp}.json"

            with open(current_file_name, 'w') as f:
                json.dump(data, f, indent=4)
            print(f"Dumped to: {current_file_name}")

        next_page_token = data.get('nextPageToken')
        if not next_page_token:
            break
        page_count += 1

    return results


In [11]:
parameters: Dict[str, str] = {
    "part": "snippet",
    "chart": "mostPopular",
    "regionCode": "IN",
}

url = get_url("/videos", parameters)
response = get_response("/videos", parameters, dump=True, max_pages=2)

Dumping page 1...
Dumped to: responses/raw/_videos_response_page_1_2025_08_15_11_31_11.json
Dumping page 2...
Dumped to: responses/raw/_videos_response_page_2_2025_08_15_11_31_11.json


In [12]:
def extract_field_types(data: List[Dict[str, Any]]) -> List[Any]:
    """Extracts field names and their types from a list of dictionaries."""
    def get_types(obj) -> Any:
        if isinstance(obj, dict):
            fields = {}
            for k, v in obj.items():
                if isinstance(v, dict):
                    fields[k] = get_types(v)
                elif isinstance(v, list) and v:
                    if isinstance(v[0], dict):
                        fields[k] = [get_types(v[0])]
                    else:
                        fields[k] = [type(v[0]).__name__]
                else:
                    fields[k] = type(v).__name__ if v is not None else None
            return fields
        return type(obj).__name__
    return [get_types(item) for item in data]


In [13]:
fields = extract_field_types(data=response)

In [14]:
pprint(fields)

[{'etag': 'str',
  'items': [{'etag': 'str',
             'id': 'str',
             'kind': 'str',
             'snippet': {'categoryId': 'str',
                         'channelId': 'str',
                         'channelTitle': 'str',
                         'defaultAudioLanguage': 'str',
                         'description': 'str',
                         'liveBroadcastContent': 'str',
                         'localized': {'description': 'str', 'title': 'str'},
                         'publishedAt': 'str',
                         'tags': ['str'],
                         'thumbnails': {'default': {'height': 'int',
                                                    'url': 'str',
                                                    'width': 'int'},
                                        'high': {'height': 'int',
                                                 'url': 'str',
                                                 'width': 'int'},
                                     

In [15]:
# using a id from the response to get more details about a video
video_id = response[0]['items'][0]['id']
video_details_params = {
    "part": "snippet,contentDetails,statistics",
    "id": video_id
}
response_details = get_response("/videos", video_details_params, dump=True, max_pages=1, base_file_name="video_response")

Dumping page 1...
Dumped to: responses/raw/video_response_response_page_1_2025_08_15_11_31_14.json


In [16]:
extract_field_types(data=response_details)

[{'kind': 'str',
  'etag': 'str',
  'items': [{'kind': 'str',
    'etag': 'str',
    'id': 'str',
    'snippet': {'publishedAt': 'str',
     'channelId': 'str',
     'title': 'str',
     'description': 'str',
     'thumbnails': {'default': {'url': 'str', 'width': 'int', 'height': 'int'},
      'medium': {'url': 'str', 'width': 'int', 'height': 'int'},
      'high': {'url': 'str', 'width': 'int', 'height': 'int'},
      'standard': {'url': 'str', 'width': 'int', 'height': 'int'},
      'maxres': {'url': 'str', 'width': 'int', 'height': 'int'}},
     'channelTitle': 'str',
     'tags': ['str'],
     'categoryId': 'str',
     'liveBroadcastContent': 'str',
     'localized': {'title': 'str', 'description': 'str'},
     'defaultAudioLanguage': 'str'},
    'contentDetails': {'duration': 'str',
     'dimension': 'str',
     'definition': 'str',
     'caption': 'str',
     'licensedContent': 'bool',
     'contentRating': {},
     'projection': 'str'},
    'statistics': {'viewCount': 'str',
   

### We want to automate the process of fetching trending videos and their details periodically

The most common entities would be video, trending_list
- for the videos we may want to fetch details like title, description, and statistics including likes, comments, duration, and view count.
- we may also need some id for each video
- for the trending list, we may want to fetch details like the list of video ids, titles, and thumbnails, and maybe links to the videos.
- we can add category in the trending list table.Also the date of first appearance.
- when updating the list periodically instead of discarding the deleted entries, we may store the video ids, appearance date, duration of trending days, link to video in a separate table.
- we may include a published on date for each video. Not to be confused with first trend appearance date, we may need to rename the columns.
- we can also include the language of the video.

~~ok let's work with that~~
No, let's pause at that. First let's see properly what is available to use.
we must go fields -> er, not er -> fields

In [2]:
import pandas as pd

In [4]:
response_details = json.load(open("../data/raw/extract/video_details_batch_1_response_page_1_2025_08_16_10_36_41.json", "r"))
response = json.load(open("../data/raw/extract/trend_list_response_page_1_2025_08_16_10_36_39.json", "r"))

In [5]:
video_fields = pd.json_normalize(response_details, record_path=['items'], meta=['kind', 'etag', ['pageInfo', 'totalResults']], meta_prefix='root_')

In [6]:
video_fields.columns

Index(['kind', 'etag', 'id', 'snippet.publishedAt', 'snippet.channelId',
       'snippet.title', 'snippet.description',
       'snippet.thumbnails.default.url', 'snippet.thumbnails.default.width',
       'snippet.thumbnails.default.height', 'snippet.thumbnails.medium.url',
       'snippet.thumbnails.medium.width', 'snippet.thumbnails.medium.height',
       'snippet.thumbnails.high.url', 'snippet.thumbnails.high.width',
       'snippet.thumbnails.high.height', 'snippet.thumbnails.standard.url',
       'snippet.thumbnails.standard.width',
       'snippet.thumbnails.standard.height', 'snippet.thumbnails.maxres.url',
       'snippet.thumbnails.maxres.width', 'snippet.thumbnails.maxres.height',
       'snippet.channelTitle', 'snippet.tags', 'snippet.categoryId',
       'snippet.liveBroadcastContent', 'snippet.localized.title',
       'snippet.localized.description', 'snippet.defaultAudioLanguage',
       'contentDetails.duration', 'contentDetails.dimension',
       'contentDetails.definitio

In [7]:
video_fields

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,snippet.defaultLanguage,root_kind,root_etag,root_pageInfo.totalResults
0,youtube#video,FTNy7gj2Zd__OtWFRXCCmfbL1Ro,tW3n8r7BxTY,2025-08-14T14:01:31Z,UCTVqVKB55ABOHWieN2thMsw,A.R. Rahman - Maa Tujhe Salaam (Lyrics) Vande ...,🎶 A.R. Rahman - Maa Tujhe Salaam\nListen Here:...,https://i.ytimg.com/vi/tW3n8r7BxTY/default.jpg,120,90,...,True,rectangular,782353,6768,0,354,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
1,youtube#video,pbsBDXVN551tMKkrjbydiknYfcg,vEXd4Ke7byM,2025-08-15T08:39:20Z,UC-LOdiPoxninevJ0DkleCLg,"Thama Teaser Announcement | Ayushmann, Rashmik...",Independence Day Special!\nThe No. 1 Hindi fil...,https://i.ytimg.com/vi/vEXd4Ke7byM/default.jpg,120,90,...,False,rectangular,3913802,59582,0,5218,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
2,youtube#video,_gI-N0kLob-lj9Ho1HUeh4F7NEc,zx8Qp3ba1lg,2025-08-15T06:30:35Z,UCX8pnu3DYUnx8qy8V_c6oHg,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,https://i.ytimg.com/vi/zx8Qp3ba1lg/default.jpg,120,90,...,True,rectangular,2032962,111138,0,9642,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
3,youtube#video,GInir1g9PrIk5TjiUCjeT64tt-w,gLdWNFFZHZE,2025-08-14T22:30:06Z,UCE4Fzpv86mA2LqxiX64VLQw,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,https://i.ytimg.com/vi/gLdWNFFZHZE/default.jpg,120,90,...,True,rectangular,549274,5941,0,299,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
4,youtube#video,kHAYj4NFmzODsm_c1Cf3XKcRDiI,QrWh3Ww3Zn0,2025-08-15T04:30:30Z,UCYBAnESPQtjTWSYsVxN9hJA,"Raghu Dakat | Official Teaser | Dev, Idhika, A...",বিদ্রোহ যার শিরায়…প্রতিশোধ যার রক্তে \nরঘু ডা...,https://i.ytimg.com/vi/QrWh3Ww3Zn0/default.jpg,120,90,...,True,rectangular,358962,26177,0,2640,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
5,youtube#video,ykYs8ubhk3Fk156vXhQnCcu-hyc,KKEgIWIwzCQ,2025-08-15T15:53:01Z,UCe31NPEeRGO0hcznx6Tdb-g,OG vs NG: Legacies Clash – Final Day | 79th In...,This is it. No more second chances. \nThe fina...,https://i.ytimg.com/vi/KKEgIWIwzCQ/default.jpg,120,90,...,False,rectangular,1133790,35247,0,101,en,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
6,youtube#video,WOasnCkRn_BDqFDKw8GEqkpKuBA,CdM7bsa4m6E,2025-08-14T05:30:04Z,UC9mHdvaR0msqacTXgdxyCRw,Har Karam Apna Karenge | Dil Diya Hai Jaan Bhi...,"Song Details :\nStar Cast: Dilip Kumar, Nutan,...",https://i.ytimg.com/vi/CdM7bsa4m6E/default.jpg,120,90,...,True,rectangular,1457993,6536,0,236,en-IN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
7,youtube#video,rQvloP0CyfTr2ProZc3alRXqwus,v1FAaUxF0kg,2025-08-12T05:20:04Z,UCGqvJPRcv7aVFun-eTsatcA,Teaser | Akshay Kumar | Arshad Warsi | Subhash...,"Pehli baar courtroom main do do Jolly, ab hoga...",https://i.ytimg.com/vi/v1FAaUxF0kg/default.jpg,120,90,...,True,rectangular,28932527,369215,0,10683,en,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
8,youtube#video,NHHMrql2rsl41EvYpHJhkBNlm04,YaWCQASns_o,2025-08-15T23:17:32Z,UCI1vyXBgX3bruwvChLMNxjQ,NON STOP RANK PUSH TO #1 | CONQUEROR OR WHAT |...,INSTAGRAM - https://www.instagram.com/lolzzzga...,https://i.ytimg.com/vi/YaWCQASns_o/default.jpg,120,90,...,True,rectangular,622623,40226,0,87,en-GB,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25
9,youtube#video,_v-kbIfvWjBvJx7DOq9jBfoYIus,98OQf963hLc,2025-08-14T09:30:06Z,UCnR-M6jXxqmK4UmeU3_5ISw,Jalwa Jalwa - Aye Watan Aye Watan| Independenc...,Jalwa Jalwa - Aye Watan Aye Watan| Independenc...,https://i.ytimg.com/vi/98OQf963hLc/default.jpg,120,90,...,True,rectangular,822243,5480,0,191,NaN,youtube#videoListResponse,PiFnbm7KSLaPknCMUH1OK7UEMQ0,25


In [8]:
playlist_fields = pd.json_normalize(response, record_path=['items'], meta=['etag', ['pageInfo', 'totalResults'], ['pageInfo', 'resultsPerPage'], 'nextPageToken', 'prevPageToken'], meta_prefix='root_', errors='ignore')

In [9]:
playlist_fields

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,snippet.categoryId,snippet.liveBroadcastContent,snippet.localized.title,snippet.localized.description,snippet.defaultAudioLanguage,root_etag,root_pageInfo.totalResults,root_pageInfo.resultsPerPage,root_nextPageToken,root_prevPageToken
0,youtube#video,ydTTQnPrnViXOZP2AwuTN6QUqcg,tW3n8r7BxTY,2025-08-14T14:01:31Z,UCTVqVKB55ABOHWieN2thMsw,A.R. Rahman - Maa Tujhe Salaam (Lyrics) Vande ...,🎶 A.R. Rahman - Maa Tujhe Salaam\nListen Here:...,https://i.ytimg.com/vi/tW3n8r7BxTY/default.jpg,120,90,...,22,none,A.R. Rahman - Maa Tujhe Salaam (Lyrics) Vande ...,🎶 A.R. Rahman - Maa Tujhe Salaam\nListen Here:...,ta,FAPHwboiB69imiE3vZE-iTtNQyc,200,5,CAUQAA,NaN
1,youtube#video,hs7ac63cANpoUEyqubfVWQiV9UU,vEXd4Ke7byM,2025-08-15T08:39:20Z,UC-LOdiPoxninevJ0DkleCLg,"Thama Teaser Announcement | Ayushmann, Rashmik...",Independence Day Special!\nThe No. 1 Hindi fil...,https://i.ytimg.com/vi/vEXd4Ke7byM/default.jpg,120,90,...,24,none,"Thama Teaser Announcement | Ayushmann, Rashmik...",Independence Day Special!\nThe No. 1 Hindi fil...,en-IN,FAPHwboiB69imiE3vZE-iTtNQyc,200,5,CAUQAA,NaN
2,youtube#video,-3N2QvHX7l8H0fGP2ket5QeWtMQ,zx8Qp3ba1lg,2025-08-15T06:30:35Z,UCX8pnu3DYUnx8qy8V_c6oHg,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,https://i.ytimg.com/vi/zx8Qp3ba1lg/default.jpg,120,90,...,20,none,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,en-IN,FAPHwboiB69imiE3vZE-iTtNQyc,200,5,CAUQAA,NaN
3,youtube#video,EEhguJ4EpFwLCQz0ueNaWH8b97I,gLdWNFFZHZE,2025-08-14T22:30:06Z,UCE4Fzpv86mA2LqxiX64VLQw,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,https://i.ytimg.com/vi/gLdWNFFZHZE/default.jpg,120,90,...,10,none,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,hi,FAPHwboiB69imiE3vZE-iTtNQyc,200,5,CAUQAA,NaN
4,youtube#video,swNC42E0fm8aYwJbEnH-wKaK1Wk,QrWh3Ww3Zn0,2025-08-15T04:30:30Z,UCYBAnESPQtjTWSYsVxN9hJA,"Raghu Dakat | Official Teaser | Dev, Idhika, A...",বিদ্রোহ যার শিরায়…প্রতিশোধ যার রক্তে \nরঘু ডা...,https://i.ytimg.com/vi/QrWh3Ww3Zn0/default.jpg,120,90,...,1,none,"Raghu Dakat | Official Teaser | Dev, Idhika, A...",বিদ্রোহ যার শিরায়…প্রতিশোধ যার রক্তে \nরঘু ডা...,en,FAPHwboiB69imiE3vZE-iTtNQyc,200,5,CAUQAA,NaN


In [10]:
playlist_fields.columns

Index(['kind', 'etag', 'id', 'snippet.publishedAt', 'snippet.channelId',
       'snippet.title', 'snippet.description',
       'snippet.thumbnails.default.url', 'snippet.thumbnails.default.width',
       'snippet.thumbnails.default.height', 'snippet.thumbnails.medium.url',
       'snippet.thumbnails.medium.width', 'snippet.thumbnails.medium.height',
       'snippet.thumbnails.high.url', 'snippet.thumbnails.high.width',
       'snippet.thumbnails.high.height', 'snippet.thumbnails.standard.url',
       'snippet.thumbnails.standard.width',
       'snippet.thumbnails.standard.height', 'snippet.thumbnails.maxres.url',
       'snippet.thumbnails.maxres.width', 'snippet.thumbnails.maxres.height',
       'snippet.channelTitle', 'snippet.tags', 'snippet.categoryId',
       'snippet.liveBroadcastContent', 'snippet.localized.title',
       'snippet.localized.description', 'snippet.defaultAudioLanguage',
       'root_etag', 'root_pageInfo.totalResults',
       'root_pageInfo.resultsPerPage', 'root

In [11]:
# Intersection and Comparison of Video and Playlist Fields
common_columns = video_fields.columns.intersection(playlist_fields.columns)
common_columns

Index(['kind', 'etag', 'id', 'snippet.publishedAt', 'snippet.channelId',
       'snippet.title', 'snippet.description',
       'snippet.thumbnails.default.url', 'snippet.thumbnails.default.width',
       'snippet.thumbnails.default.height', 'snippet.thumbnails.medium.url',
       'snippet.thumbnails.medium.width', 'snippet.thumbnails.medium.height',
       'snippet.thumbnails.high.url', 'snippet.thumbnails.high.width',
       'snippet.thumbnails.high.height', 'snippet.thumbnails.standard.url',
       'snippet.thumbnails.standard.width',
       'snippet.thumbnails.standard.height', 'snippet.thumbnails.maxres.url',
       'snippet.thumbnails.maxres.width', 'snippet.thumbnails.maxres.height',
       'snippet.channelTitle', 'snippet.tags', 'snippet.categoryId',
       'snippet.liveBroadcastContent', 'snippet.localized.title',
       'snippet.localized.description', 'snippet.defaultAudioLanguage',
       'root_etag', 'root_pageInfo.totalResults'],
      dtype='object')

In [12]:
# Unique columns
video_fields = video_fields.drop(columns=common_columns)
playlist_fields = playlist_fields.drop(columns=common_columns)
print("Video Fields Unique Columns:")
print(video_fields.columns)
print("Playlist Fields Unique Columns:")
print(playlist_fields.columns)

Video Fields Unique Columns:
Index(['contentDetails.duration', 'contentDetails.dimension',
       'contentDetails.definition', 'contentDetails.caption',
       'contentDetails.licensedContent', 'contentDetails.projection',
       'statistics.viewCount', 'statistics.likeCount',
       'statistics.favoriteCount', 'statistics.commentCount',
       'snippet.defaultLanguage', 'root_kind'],
      dtype='object')
Playlist Fields Unique Columns:
Index(['root_pageInfo.resultsPerPage', 'root_nextPageToken',
       'root_prevPageToken'],
      dtype='object')


This gives us an idea of the specific columns that can be used in er

id -> This is our unique video_id. Must be in videos table

snippet.title, snippet.description, snippet.channelTitle, snippet.categoryId -> These are all core attributes of a video and belong directly in our videos table.

contentDetails.duration -> It will be a column in our videos table.

statistics.viewCount, statistics.likeCount, statistics.commentCount -> These are the metrics we're interested in. They also belong in our videos table.

For linking trend list to videos table, the trend list may contain
a. video_id -> This will be the foreign key in the trend list table linking back to the videos table.


In [ ]:
# Make a two pass api call 

parameters_trend_list: Dict[str, str] = {
    "part": "snippet",
    "chart": "mostPopular",
    "regionCode": "IN",
}

video_details_params = {
    "part": "snippet,contentDetails,statistics",
    "id": video_id
}

get_trend_list = get_response("/videos", parameters_trend_list, dump=True, max_pages=5, base_file_name="trend_list_response")
videos = response_details[0]['items']


for page in get_trend_list:
    if page.get('items'):
        for item in page['items']:
            video_id = item['id']
            video_details_params['id'] = video_id
            video_details = get_response("/videos", video_details_params, dump=True, max_pages=1, base_file_name=f"video_details_{video_id}")
            if video_details[0].get('items'):
                videos.append(video_details[0]['items'][0])


Dumping page 1...
Dumped to: responses/raw/trend_list_response_response_page_1_2025_08_15_12_02_42.json
Dumping page 2...
Dumped to: responses/raw/trend_list_response_response_page_2_2025_08_15_12_02_43.json
Dumping page 3...
Dumped to: responses/raw/trend_list_response_response_page_3_2025_08_15_12_02_43.json
Dumping page 4...
Dumped to: responses/raw/trend_list_response_response_page_4_2025_08_15_12_02_43.json
Dumping page 5...
Dumped to: responses/raw/trend_list_response_response_page_5_2025_08_15_12_02_44.json
Dumping page 1...
Dumped to: responses/raw/video_details_tW3n8r7BxTY_response_page_1_2025_08_15_12_02_44.json
Dumping page 1...
Dumped to: responses/raw/video_details_fdWnfzsx-ks_response_page_1_2025_08_15_12_02_44.json
Dumping page 1...
Dumped to: responses/raw/video_details_8_4kQftrqiU_response_page_1_2025_08_15_12_02_44.json
Dumping page 1...
Dumped to: responses/raw/video_details_CdM7bsa4m6E_response_page_1_2025_08_15_12_02_45.json
Dumping page 1...
Dumped to: responses/r

In [5]:
import pandas as pd

In [6]:
complete_details = json.load(open("../data/raw/extract/complete_extraction_2025_08_16_16_49_49.json", "r"))
response = complete_details["trend_list"] 
response_details = complete_details["video_details"][0]


In [7]:
trend_ids = pd.json_normalize(
    response,
    record_path=['items'],
    meta=['etag', 'nextPageToken', 'prevPageToken', 'kind'],
    meta_prefix='root_',
    errors='ignore'
)[['id', 'root_kind', 'root_etag', 'root_nextPageToken', 'root_prevPageToken']]

trend_ids = trend_ids.rename(columns={'id': 'video_id'})

In [8]:
trend_ids.columns

Index(['video_id', 'root_kind', 'root_etag', 'root_nextPageToken',
       'root_prevPageToken'],
      dtype='object')

In [9]:
response_details

{'kind': 'youtube#videoListResponse',
 'etag': 'L2ED9Fk7hgi8_Sd1M2sWTTQa9gw',
 'items': [{'kind': 'youtube#video',
   'etag': 'FcIAmuuX_5n873u59rJGbUQmbJg',
   'id': 'gLdWNFFZHZE',
   'snippet': {'publishedAt': '2025-08-14T22:30:06Z',
    'channelId': 'UCE4Fzpv86mA2LqxiX64VLQw',
    'title': '🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashtami Special 2025 | Nonstop Radha Krishna Songs 🙏',
    'description': '🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashtami Special 2025 | Nonstop Radha Krishna Songs 🙏\n\n00:00 - Krishnaya Vasudevaya\n09:10 - Madhurashtakam\n14:30 - Achyutam Keshavam\n24:30 - Hare Krishna Hare Rama\n33:46 - O Kanha Ab To Murli Ki\n37:33 - Govinda Bolo Hari Gopal Bolo\n41:53 - Shri Krishna Govind Hare Murari\n45:54 - Maadhav Madhusudana\n51:24 - Radhe Govind Krishna Murari\n55:41 - Radhika Manohara\n\n🕉 जय श्री राधे कृष्णा 🌸\nLet these bhajans bring blessings, joy, and devotion into your life. 🙏💫\n\nTitle - Achyutam Keshavam\nSinger - Minakshi Majumdar\nAlbum - Bhakti Sange

In [10]:
videos = response_details['items']

In [11]:
videos = pd.json_normalize(videos)


In [12]:
videos.columns

Index(['kind', 'etag', 'id', 'snippet.publishedAt', 'snippet.channelId',
       'snippet.title', 'snippet.description',
       'snippet.thumbnails.default.url', 'snippet.thumbnails.default.width',
       'snippet.thumbnails.default.height', 'snippet.thumbnails.medium.url',
       'snippet.thumbnails.medium.width', 'snippet.thumbnails.medium.height',
       'snippet.thumbnails.high.url', 'snippet.thumbnails.high.width',
       'snippet.thumbnails.high.height', 'snippet.thumbnails.standard.url',
       'snippet.thumbnails.standard.width',
       'snippet.thumbnails.standard.height', 'snippet.thumbnails.maxres.url',
       'snippet.thumbnails.maxres.width', 'snippet.thumbnails.maxres.height',
       'snippet.channelTitle', 'snippet.tags', 'snippet.categoryId',
       'snippet.liveBroadcastContent', 'snippet.localized.title',
       'snippet.localized.description', 'snippet.defaultAudioLanguage',
       'contentDetails.duration', 'contentDetails.dimension',
       'contentDetails.definitio

In [13]:
videos = videos.rename(columns={
    'snippet.publishedAt': 'published_at',
    'snippet.channelId': 'channel_id',
    'snippet.thumbnails.default.url': 'thumbnail_url',
    'snippet.channelTitle': 'channel_title',
    'snippet.tags': 'tags',
    'snippet.categoryId': 'category_id',
    'snippet.localized.title': 'localized_title',
    'snippet.localized.description': 'localized_description',
    'contentDetails.duration': 'duration',
    'contentDetails.dimension': 'dimension',
    'contentDetails.definition': 'definition',
    'statistics.viewCount': 'view_count',
    'statistics.likeCount': 'like_count',
    'statistics.dislikeCount': 'dislike_count',
    'statistics.favoriteCount': 'favorite_count',
    'statistics.commentCount': 'comment_count',
    'snippet.defaultAudioLanguage': 'default_language'
})

In [14]:
videos_clean = videos.drop(columns=[col for col in videos.columns if col.startswith(('contentDetails', 'snippet', 'statistics'))])\
                                .drop(columns=['dimension', 'tags', 'definition'])

In [15]:
videos_clean.columns

Index(['kind', 'etag', 'id', 'published_at', 'channel_id', 'thumbnail_url',
       'channel_title', 'category_id', 'localized_title',
       'localized_description', 'default_language', 'duration', 'view_count',
       'like_count', 'favorite_count', 'comment_count'],
      dtype='object')

In [16]:
trend_ids["first_appearance_date"] = pd.to_datetime('today').strftime('%Y-%m-%d')
# trend_ids["last_appearance_date"] = pd.to_datetime('today').strftime('%Y-%m-%d') this will be during loading only

In [17]:

trend_ids.rename(columns={'video_id': 'id'
                          , 'root_kind': 'kind'
                          , 'root_etag': 'etag'
                          , 'root_nextPageToken': 'nextPageToken'
                          , 'root_prevPageToken': 'prevPageToken'
                          }                 
                 , inplace=True)

In [18]:
trend_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     55 non-null     object
 1   kind                   55 non-null     object
 2   etag                   55 non-null     object
 3   nextPageToken          55 non-null     object
 4   prevPageToken          50 non-null     object
 5   first_appearance_date  55 non-null     object
dtypes: object(6)
memory usage: 2.7+ KB


In [19]:
import pyarrow as pa

In [20]:
trend_ids = trend_ids.astype(
    {
        'id': 'string[pyarrow]',
        'etag': 'string[pyarrow]',
        'nextPageToken': 'string[pyarrow]',
        'prevPageToken': 'string[pyarrow]',
        'kind': 'category'
    }
)
trend_ids["first_appearance_date"] = pd.to_datetime(trend_ids["first_appearance_date"], errors='coerce', format='%Y-%m-%d')\
    .astype(pd.ArrowDtype(pa.date32()))


In [21]:
trend_ids['first_appearance_date'].dtype

date32[day][pyarrow]

In [22]:
trend_ids.head()

,id,kind,etag,nextPageToken,prevPageToken,first_appearance_date
0,gLdWNFFZHZE,youtube#videoListResponse,E3CNO702CvvsSn8EuQduGSIKOkk,CAUQAA,<NA>,2025-08-16
1,3MfsZFAeNO8,youtube#videoListResponse,E3CNO702CvvsSn8EuQduGSIKOkk,CAUQAA,<NA>,2025-08-16
2,zx8Qp3ba1lg,youtube#videoListResponse,E3CNO702CvvsSn8EuQduGSIKOkk,CAUQAA,<NA>,2025-08-16
3,tW3n8r7BxTY,youtube#videoListResponse,E3CNO702CvvsSn8EuQduGSIKOkk,CAUQAA,<NA>,2025-08-16
4,vEXd4Ke7byM,youtube#videoListResponse,E3CNO702CvvsSn8EuQduGSIKOkk,CAUQAA,<NA>,2025-08-16


In [23]:
trend_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype               
---  ------                 --------------  -----               
 0   id                     55 non-null     string              
 1   kind                   55 non-null     category            
 2   etag                   55 non-null     string              
 3   nextPageToken          55 non-null     string              
 4   prevPageToken          50 non-null     string              
 5   first_appearance_date  55 non-null     date32[day][pyarrow]
dtypes: category(1), date32[day][pyarrow](1), string(4)
memory usage: 4.9 KB


In [24]:
videos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   kind                   50 non-null     object
 1   etag                   50 non-null     object
 2   id                     50 non-null     object
 3   published_at           50 non-null     object
 4   channel_id             50 non-null     object
 5   thumbnail_url          50 non-null     object
 6   channel_title          50 non-null     object
 7   category_id            50 non-null     object
 8   localized_title        50 non-null     object
 9   localized_description  50 non-null     object
 10  default_language       44 non-null     object
 11  duration               50 non-null     object
 12  view_count             50 non-null     object
 13  like_count             48 non-null     object
 14  favorite_count         50 non-null     object
 15  comment_count          50

In [25]:
videos_clean['published_at'] = pd.to_datetime(videos_clean['published_at'], errors='coerce').dt.date #Get date

videos_clean['published_at'] = videos_clean['published_at'].astype(pd.ArrowDtype(pa.date32())) #Convert

videos_clean['duration'] = pd.to_timedelta(videos_clean['duration'], errors='coerce')\
                            .dt.total_seconds()\
                            .astype('float32[pyarrow]') #Calculate duration in seconds and convert


In [26]:
videos_clean = videos_clean.astype(
    {
        'kind': 'category',
        'etag': 'string[pyarrow]',
        'id': 'string[pyarrow]',
        'channel_id': 'string[pyarrow]',
        'thumbnail_url': 'string[pyarrow]',
        'channel_title': 'string[pyarrow]',
        'category_id': 'category',
        'localized_title': 'string[pyarrow]',
        'localized_description': 'string[pyarrow]',
        'default_language': 'category',
        'view_count': 'Int64[pyarrow]',
        'like_count': 'Int64[pyarrow]',
        'comment_count': 'Int32[pyarrow]',
    },
)
videos_clean["favorite_count"] = videos_clean["favorite_count"].astype(pd.SparseDtype("int32", 0))

videos_clean.head()

,kind,etag,id,published_at,channel_id,thumbnail_url,channel_title,category_id,localized_title,localized_description,default_language,duration,view_count,like_count,favorite_count,comment_count
0,youtube#video,FcIAmuuX_5n873u59rJGbUQmbJg,gLdWNFFZHZE,2025-08-14,UCE4Fzpv86mA2LqxiX64VLQw,https://i.ytimg.com/vi/gLdWNFFZHZE/default.jpg,Divine Bhajans,10,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,🌺 Top 10 Krishna Bhajans 🎶 | Krishna Janmashta...,hi,3774.0,1404295,8845,0,460
1,youtube#video,uZlK1l4dY2en6D15SIrrB0sbGfQ,3MfsZFAeNO8,2025-08-16,UC3jMepkLKF8y4iiwWmAB3RA,https://i.ytimg.com/vi/3MfsZFAeNO8/default.jpg,Zee Studios,1,THE BENGAL FILES Official Trailer | Vivek Agni...,From the makers of The Kashmir Files and The T...,en,213.0,463518,68673,0,7534
2,youtube#video,7xTdinwdW5_F3FkttErcDam2W98,zx8Qp3ba1lg,2025-08-15,UCX8pnu3DYUnx8qy8V_c6oHg,https://i.ytimg.com/vi/zx8Qp3ba1lg/default.jpg,Techno Gamerz,20,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,ZOMBIE BORDER PATROL | QUARANTINE ZONE : THE L...,en-IN,2819.0,2622196,126430,0,10740
3,youtube#video,KqzRiFAzz6p-S0DJ3hTAX-9VfN8,tW3n8r7BxTY,2025-08-14,UCTVqVKB55ABOHWieN2thMsw,https://i.ytimg.com/vi/tW3n8r7BxTY/default.jpg,seventyskye,22,A.R. Rahman - Maa Tujhe Salaam (Lyrics) Vande ...,🎶 A.R. Rahman - Maa Tujhe Salaam\nListen Here:...,ta,376.0,822034,6998,0,360
4,youtube#video,ty9oqrRILPG2vE4yOA8jNkNcUzw,vEXd4Ke7byM,2025-08-15,UC-LOdiPoxninevJ0DkleCLg,https://i.ytimg.com/vi/vEXd4Ke7byM/default.jpg,Maddock Films,24,"Thama Teaser Announcement | Ayushmann, Rashmik...",Independence Day Special!\nThe No. 1 Hindi fil...,en-IN,43.0,4603624,78256,0,6095


In [27]:
videos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype               
---  ------                 --------------  -----               
 0   kind                   50 non-null     category            
 1   etag                   50 non-null     string              
 2   id                     50 non-null     string              
 3   published_at           50 non-null     date32[day][pyarrow]
 4   channel_id             50 non-null     string              
 5   thumbnail_url          50 non-null     string              
 6   channel_title          50 non-null     string              
 7   category_id            50 non-null     category            
 8   localized_title        50 non-null     string              
 9   localized_description  50 non-null     string              
 10  default_language       44 non-null     category            
 11  duration               50 non-null     float[py

/tmp/ipykernel_30998/1690785849.py:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  videos_clean.info()


In [28]:
videos_clean['favorite_count'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 50 entries, 0 to 49
Series name: favorite_count
Non-Null Count  Dtype           
--------------  -----           
50 non-null     Sparse[int32, 0]
dtypes: Sparse[int32, 0](1)
memory usage: 532.0 bytes
